In [1]:
class TSP:
    
    ## Initialization ##
    def __init__(self, cities):
        self.cities = cities
        self.n = len(self.cities)
        self.dist = [[0] * self.n for i in range(self.n)]
        pass
    
    
    ## Calculate the distance between two cities #
    def twoCitiesDistance(self, city1, city2):
        return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
        pass
    
    
    ## Calculate the total distance of a tour ##
    def totalDistance(self, tour):

        start = tour
        end = tour[1:]
        end.append(tour[0])
        distance = 0
    
        for i, j in zip(start,end):
            distance += self.dist[i][j]
        
        return distance
        pass
    
    
    ## Generate a distance matrix ##
    def distanceMatrix(self):
        for i in range(self.n):
            for j in range(i, self.n):
                self.dist[i][j] = self.dist[j][i] = self.twoCitiesDistance(self.cities[i], self.cities[j])
        pass
    
    
    ## Judge whether two edges (p1---p2, p3---p4) are overlapped ##
    def isOverlap(self, p1, p2, p3, p4):
 
        x1 = self.cities[p1][0]
        y1 = self.cities[p1][1]
        x2 = self.cities[p2][0]
        y2 = self.cities[p2][1]
        x3 = self.cities[p3][0]
        y3 = self.cities[p3][1]
        x4 = self.cities[p4][0]
        y4 = self.cities[p4][1]
        
        temp = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
        if temp != 0:
            # coordinate of the intersection point (x,y)
            x = ((x1*y2-y1*x2) * (x3-x4) - (x1-x2) * (x3*y4-y3*x4)) / temp
            y = ((x1*y2-y1*x2) * (y3-y4) - (y1-y2) * (x3*y4-y3*x4)) / temp
            # check if intersect. point is on two edges
            if min(x1,x2)<x<max(x1,x2) and min(y1,y2)<y<max(y1,y2) and min(x3,x4)<x<max(x3,x4) and min(y3,y4)<y<max(y3,y4):
                return True
        return False
        pass
    
    
    ## Un-overlap all edges ##
    def processOverlap(self, tour):
        
        overlap_flag = 1
        
        while overlap_flag == 1:
        
            overlap_flag = 0
            new_tour = copy.deepcopy(tour)
            
            # List all the edges
            edges = []
            for i in range(self.n-1):
                edges.append([tour[i], tour[i+1]])
            edges.append([tour[self.n-1], tour[0]])
 
            # check overlapped edges
            for i in range(len(edges)-1):
                for j in range(i+1, len(edges)):
                    # edge 1: p1<->p2
                    # edge 2: p3<->p4
                    p1, p2, p3, p4 = edges[i][0], edges[i][1], edges[j][0], edges[j][1]
                    old_dist = self.dist[p1][p2] + self.dist[p3][p4]
                
                    # if edge 1 and edge 2 overlap, switch. (p1<->p3; p2<->p4)
                    if self.isOverlap(p1, p2, p3, p4):
                        new_dist = self.dist[p1][p3] + self.dist[p2][p4]
                        if new_dist < old_dist:
                            new_tour = copy.deepcopy(tour)
                            new_tour[i+1:j+1] = new_tour[j:i:-1]
                            overlap_flag = 1
            tour = new_tour 
            
        return tour
        pass

                          
    ## greedy algorithm + 2-opt ##
    def greedy(self):

        best_tour = []
        shortest_dist = float('inf')
        
        for current_city in range(self.n):
            unvisited_cities = set(list(range(0,current_city)) + list(range(current_city+1,self.n)))
            tour = [current_city]

            while unvisited_cities:
                next_city = min(unvisited_cities, key=lambda city: self.dist[current_city][city])
                unvisited_cities.remove(next_city)
                tour.append(next_city)
                current_city = next_city
            
            tour = self.processOverlap(tour)
                
            distance = self.totalDistance(tour)
            if distance < shortest_dist:
                shortest_dist = distance
                best_tour = tour
                
        return best_tour
        pass
        
    
    def solve(self):  
        self.distanceMatrix()
        best_tour = self.greedy()   
        return best_tour, self.totalDistance(best_tour)  

In [2]:
if __name__ == '__main__':
    
    import sys
    import math
    import copy
    import numpy as np
    from common import print_tour, read_input

In [3]:
for i in range(5):
    
    file_name = 'input_'+str(i)+'.csv'
    input_data = read_input(file_name)
    solution = TSP(input_data)
    best_tour, shortest_dist = solution.solve()
    
    #print('===== Best Tour for', file_name, '=====')
    #print_tour(best_tour)
    print('===== Shortest Distance for', file_name, '=====\n', shortest_dist)

===== Shortest Distance for input_0.csv =====
 3418.101599132713
===== Shortest Distance for input_1.csv =====
 3832.2900939051992
===== Shortest Distance for input_2.csv =====
 4664.521526470755
===== Shortest Distance for input_3.csv =====
 8903.614511455757
===== Shortest Distance for input_4.csv =====
 11393.004980010171
